In [1]:
import lda_clust

In [2]:
x =lda_clust.simulate_data(D=200, K=3, gamma=5, command_level_topics=False, secondary_topic=False, V=10, csi=10, omega=5)

In [3]:
m = lda_clust.topic_model(W=x['w'], K=20, fixed_K = True, V=10, fixed_V=True, secondary_topic=True, 
            command_level_topics=True, H=3, fixed_H=True, gamma=1.0, eta=1.0)

In [4]:
m.random_init()

In [5]:
for _ in range(1000):
    m.resample_session_topics()

In [6]:
for _ in range(1000):
    m.resample_indicators()

In [7]:
for _ in range(1000):
    m.resample_command_topics()

In [8]:
for _ in range(10):
    m.split_merge_session(random_allocation=True)

In [9]:
m.T

array([89, 22,  5,  0,  7,  4, 34,  0,  6,  0,  0, 16,  0,  6,  0,  0,  6,
        2,  0,  3])

In [10]:
m.split_merge_command()

/Users/fraspass/Documents/github/lda_clust/lib/lda_clust/topic_model.py:918: RuntimeWarning: invalid value encountered in log
  probs += np.sum(np.log(np.add.outer(self.alpha0 + M_ast_temp - Z_temp, np.arange(self.M[doc][command] - Zjd))), axis=1)


ValueError: probabilities contain NaN

In [ ]:
self = m
import numpy as np
indices_int = np.random.choice(self.N_cumsum[-1], size=2, replace=False)

In [ ]:
indices_int

In [ ]:
## First index
d = np.argmax(np.logical_not(indices_int[0] < np.append(0,self.N_cumsum)))
j = indices_int[0] - (0 if d == 0 else self.N_cumsum[d-1])
## Second index
d_prime = np.argmax(self.N_cumsum[indices_int[1] < self.N_cumsum])
j_prime = indices_int[1] - (0 if d_prime == 0 else self.N_cumsum[d_prime-1])

In [ ]:
indices_int = np.array([1,14])
## First index
d = np.argmax(self.N_cumsum0[np.logical_not(indices_int[0] < self.N_cumsum0)])
j = indices_int[0] - self.N_cumsum0[d]
## Second index
d_prime = np.argmax(self.N_cumsum0[np.logical_not(indices_int[1] < self.N_cumsum0)])
j_prime = indices_int[1] - self.N_cumsum0[d_prime]

In [ ]:
self = m
import numpy as np
from collections import Counter

In [ ]:
print(d,j,d_prime,j_prime)

In [ ]:
self.N_cumsum

In [ ]:
indices_int

In [ ]:
d, d_prime = np.random.choice(self.D, size=2, replace=False)

In [ ]:
self.t[d] == self.t[d_prime]

In [ ]:
if np.sum(self.T == 0) < self.K:
                split = False
                t = np.min([self.t[d],self.t[d_prime]])
                t_ast = np.max([self.t[d],self.t[d_prime]])
else:
                boundary = True

In [ ]:
# Merge move
indices = np.where(np.logical_or(self.t == t, self.t == t_ast))[0]
indices = indices[np.logical_and(indices != d, indices != d_prime)]
T_prop = np.array([self.T[t] + self.T[t_ast],0])
T_temp = np.ones(2)
if self.command_level_topics:
    S_prop = np.zeros((2,self.H)); S_prop[0] = self.S[t] + self.S[t_ast]
    S_temp = np.zeros((2,self.H))
    Q = Counter(self.s[d])
    for h in Q:
        S_temp[0,h] = Q[h]
    Q = Counter(self.s[d_prime])
    for h in Q:
        S_temp[1,h] = Q[h]
else:
    W_prop = np.zeros((2,self.V)); W_prop[0] = self.W[t + (1 if self.secondary_topic else 0)] + self.W[t_ast + (1 if self.secondary_topic else 0)]
    W_temp = np.zeros((2,self.V))
    for j in self.w[d]:
        Q = Counter(self.w[d][j] if not self.secondary_topic else self.w[d][j][self.z[d][j] == 1])
        for v in Q:
            W_temp[0,v] += Q[v] 
    for j in self.w[d_prime]:    
        Q = Counter(self.w[d_prime][j] if not self.secondary_topic else self.w[d_prime][j][self.z[d_prime][j] == 1])
        for v in Q:
            W_temp[1,v] += Q[v] 
    if self.secondary_topic:
        M_ast_prop = np.zeros(2); M_ast_prop[0] = self.M_star[t] + self.M_star[t_ast]
        Z_prop = np.zeros(2); Z_prop[0] = self.Z[t] + self.Z[t_ast]
        M_ast_temp = np.zeros(2); Z_temp = np.zeros(2)
        for j in self.w[d]:
            M_ast_temp[0] += self.M[d][j]
            Z_temp[0] += np.sum(self.w[d][j])
        for j in self.w[d_prime]:
            M_ast_temp[1] += self.M[d_prime][j]
            Z_temp[1] += np.sum(self.w[d_prime][j])

In [ ]:
indices[0]

In [ ]:
from collections import Counter
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds
from sklearn.cluster import KMeans